<a href="https://colab.research.google.com/github/Oviing/nlp/blob/main/sarcasm_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import kaggle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

import spacy

import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Download Dataset from Kaggle

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 62 bytes


In [ ]:
!kaggle datasets download -d danofer/sarcasm

 93% 201M/216M [00:02<00:00, 86.6MB/s]
100% 216M/216M [00:02<00:00, 86.2MB/s]


In [ ]:
!unzip sarcasm.zip

Archive:  sarcasm.zip
  inflating: test-balanced.csv       
  inflating: test-unbalanced.csv     
  inflating: train-balanced-sarc.csv.gz  
  inflating: train-balanced-sarcasm.csv  


Open data file

In [ ]:
sarcasm = pd.read_csv('/content/train-balanced-sarcasm.csv')

In [ ]:
sarcasm.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


Creating training and test set

80% training set and 20% test set

In [ ]:
X = sarcasm[['parent_comment', 'subreddit']].values
y = sarcasm['label'].values

In [ ]:
X_train, X_test = train_test_split(sarcasm, test_size = 0.2, random_state =42,stratify = sarcasm['label'])

In [ ]:
X_train.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
928042,0,Central Illinois,ocxtitan,AdviceAnimals,1,1,0,2013-09,2013-09-10 22:39:41,"Jesus, where do you live?"
563885,0,To think - CNN used to be the acronym synonymo...,LeChuckly,POLITIC,2,2,0,2015-09,2015-09-11 15:51:39,Even The CNN Staff Is Sick Of The Wall-To-Wall...
754036,0,"But then again, you have to consider that all ...",throwitskrub8,india,1,1,0,2014-12,2014-12-08 11:58:01,agree to that part.It can also mean that gujra...
361336,0,ughhhhh,fresherthanyouuu,AskWomen,2,2,0,2015-11,2015-11-23 12:48:09,If a guy told you he doesn't use social media ...
930956,1,To ensure you're never late to the hairdressers.,markspyguy,carporn,1,1,0,2013-05,2013-05-26 05:31:32,2011 BMW Z4 GT3


In [ ]:
X_test.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
873845,1,Such a beautiful sport,parko4,funny,-4,-4,0,2014-06,2014-06-10 18:20:56,And the Oscar goes to...
279738,1,Well fuck you too.,favoritedisguise,nba,2,2,0,2016-06,2016-06-20 04:20:33,Whys everybody gotta be so mother fucking frie...
410011,1,Can't tell if,Hold_onto_yer_butts,worldnews,3,3,0,2015-11,2015-11-03 19:41:48,I feel like just giving money to get stuff don...
786465,1,Just like the HPV vaccine increases sexual act...,liewor,worldnews,2,2,0,2015-04,2015-04-24 19:49:08,Cuba has had a lung cancer vaccine for years. ...
328438,0,There goes my hope that Kubo does a Kojima as ...,Ellefied,bleach,3,3,0,2016-08,2016-08-18 12:29:47,Plus the Japanese typically do not talk shit w...


Preprocess data

Preprocesssing classes

In [ ]:
class CleanText():
    
    def __init__(self):
        return
        
    def fit(self, X, y = None):
        return self
    
    def clean_punctuation(self, x):
        
        #returns a list with tokinized words. Each string is a seperate element of the list
        x = str(x)
        w = word_tokenize(x)
        
        #removes all non alphabetic characters
        clean_list = [i for i in w if i.isalpha()]
        
        return clean_list
    
    def transform(self, X, y = None):
        
        l = []
        
        for i in X:
            
            cp = self.clean_punctuation(i)
            l.append(cp)
        
        
        return l

class Stop():
    
    def __init__(self, language):
        self.lang = language
        
    def fit(self, X, y = None):
        return self
    
    def remove_stop_words(self, x):
        cleaned_list = [w for w in x if not w in stopwords.words(self.lang)]
        return cleaned_list
    
    def transform(self, X, y = None):
        l = []
        for i in X:
            cleaned_sublist = self.remove_stop_words(i)
            l.append(cleaned_sublist)
        
        return l

class Flatten():
    def __init__(self):
        return
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
        
        flatten = [item for sublist in X for item in sublist]
        
        return flatten

class Lower():
    def __init__(self):
        return
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
        
        lower_list = []
        for i in X:
            X_actual = i
            lower = [x.lower() for x in X_actual]
            lower_list.append(lower)
        
        list_of_list_to_string = [' '.join(l) for l in lower_list]
                          
        return list_of_list_to_string

class NotSparse():
    
    def __init__(self):
        return
    def fit(self, X, y = None):
        return self
    def transform(self, X, y = None):
        return X.toarray()

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
def spacy_process(text):
    doc = nlp(text)
    
  #Tokenization and lemmatization are done with the spacy nlp pipeline commands
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)
    #print("Tokenize+Lemmatize:")
    #print(lemma_list)
    
    #Filter the stopword
    filtered_sentence =[] 
    for word in lemma_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
    
    #Remove punctuation
    punctuations="?:!.,;"
    for word in filtered_sentence:
        if word in punctuations:
            filtered_sentence.remove(word)
   # print(" ")
   # print("Remove stopword & punctuation: ")
   # print(filtered_sentence)
    listToStr = ' '.join([str(elem) for elem in filtered_sentence])
    return listToStr

Preprocess

In [ ]:
X_train_text = X_train['parent_comment'].values.tolist()
X_test_text = X_test['parent_comment'].values.tolist()

In [ ]:
pipe = Pipeline([("Remove_Punktuation", CleanText()),
                 ("Stopwords", Stop('english')),
                  ("Lower_case", Lower())])
                 #("TFIDF" , TfidfVectorizer()),
                 #("NOT_SPARSE", NotSparse())])

In [ ]:
pipe.fit(X_train_text)

Pipeline(memory=None,
         steps=[('Remove_Punktuation',
                 <__main__.CleanText object at 0x7f779f098110>),
                ('Stopwords', <__main__.Stop object at 0x7f779f098150>),
                ('Lower_case', <__main__.Lower object at 0x7f779f098190>)],
         verbose=False)

In [ ]:
pipe_training_set = pipe.transform(X_train_text)

In [ ]:
pipe_test_set = pipe.transform(X_test_text)

In [ ]:
len(pipe_training_set), len(X_train)

(808660, 808660)

In [ ]:
spacy_process(X_train_text[1])

Tokenize+Lemmatize:
['even', 'the', 'CNN', 'Staff', 'be', 'Sick', 'of', 'the', 'Wall', '-', 'To', '-', 'Wall', 'Trump', 'Coverage']
 
Remove stopword & punctuation: 
['CNN', 'Staff', 'Sick', 'Wall', '-', '-', 'Wall', 'Trump', 'Coverage']


'CNN Staff Sick Wall - - Wall Trump Coverage'

In [ ]:
doc = nlp(X_train_text[0])

In [ ]:
for token in doc:
  print(token.lemma_)

Jesus
,
where
do
-PRON-
live
?


In [ ]:
pipe_training_set = []

for i in range(0, len(X_train_text)):
  doc = X_train_text[i]
  doc_preprocessed = spacy_process(doc)
  pipe_training_set.append(doc_preprocessed)

In [ ]:
pipe_test_set = []

for i in range(0, len(X_test_text)):
  doc = X_test_text[i]
  doc_preprocessed = spacy_process(doc)
  pipe_test_set.append(doc_preprocessed)

In [ ]:
pipe_training_set[3]

'guy tell -PRON- -PRON- use social medium -PRON- head'

Create new Data Frames with preprocessed data

Comment, Subreddit, Label

In [ ]:
data = []
for i in range(0, len(pipe_training_set)):

  c = pipe_training_set[i]
  s = X_train['subreddit'].iloc[i]
  l = X_train['label'].iloc[i]

  data.append((c,s,l))

In [ ]:
X_preprocessed_train = pd.DataFrame(data, columns = ['comment', 'subreddit', 'label'])

In [ ]:
X_preprocessed_train.head()

,comment,subreddit,label
0,Jesus -PRON- live,AdviceAnimals,0
1,CNN Staff Sick Wall - - Wall Trump Coverage,POLITIC,0
2,agree -PRON- mean gujrat husband good suspect ...,india,0
3,guy tell -PRON- -PRON- use social medium -PRON...,AskWomen,0
4,2011 BMW Z4 GT3,carporn,1


In [ ]:
X_preprocessed_train.to_csv('train_reddit.csv', index= None)

In [ ]:
data = []
for i in range(0, len(pipe_test_set)):

  c = pipe_test_set[i]
  s = X_test['subreddit'].iloc[i]
  l = X_test['label'].iloc[i]

  data.append((c,s,l))

In [ ]:
X_preprocessed_test = pd.DataFrame(data, columns = ['comment', 'subreddit', 'label'])

In [ ]:
X_preprocessed_test.to_csv('test_reddit.csv', index= None)

Delete the lower part

In [ ]:
train_temp = pd.DataFrame(pipe_training_set)

In [ ]:
test_temp = pd.DataFrame(pipe_test_set)

In [ ]:
X_preprocessed_train = pd.concat([train_temp, 
                                            X_train['subreddit'], 
                                            X_train['label']], axis=1)

In [ ]:
X_preprocessed_test = pd.concat([test_temp, 
                                            X_test['subreddit'], 
                                            X_test['label']], axis=1)

In [ ]:
X_preprocessed_train.head()

,0,subreddit,label
0,jesus live,politics,0.0
1,even the cnn staff is sick of the trump coverage,nba,0.0
2,agree also mean gujrat husbands good suspectin...,nfl,0.0
3,if guy told use social media goes head,BlackPeopleTwitter,0.0
4,bmw,MaddenUltimateTeam,0.0


In [ ]:
X_preprocessed_train.to_csv('train_reddit.csv', index= None)

In [ ]:
X_preprocessed_test.to_csv('test_reddit.csv', index= None)

In [ ]:
d = pd.read_csv('/content/train_reddit.csv')

In [ ]:
d.head()

,0,subreddit,label
0,jesus live,politics,0.0
1,even the cnn staff is sick of the trump coverage,nba,0.0
2,agree also mean gujrat husbands good suspectin...,nfl,0.0
3,if guy told use social media goes head,BlackPeopleTwitter,0.0
4,bmw,MaddenUltimateTeam,0.0


In [ ]:
from sklearn.metrics.pairwise import euclidean_distances